In [ ]:
import pandas as pd
import seaborn as sns
import seaborn.objects as so
import datetime as dt
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from matplotlib.ticker import FuncFormatter
from formulaic import model_matrix, Formula
import numpy as np

## Procesamiento de datos

In [ ]:
ruta_datasets = 'data/'

In [ ]:
dataset_sube20 = pd.read_csv(ruta_datasets+'dat-ab-usos-2020.csv')
dataset_sube21 = pd.read_csv(ruta_datasets+'dat-ab-usos-2021.csv')
dataset_sube22 = pd.read_csv(ruta_datasets+'dat-ab-usos-2022.csv')
dataset_sube = pd.read_csv(ruta_datasets+'sube-2023.csv')
dataset_sube_regresion = pd.read_csv(ruta_datasets+'sube-2023-regresion.csv')
poblacion_provincias = pd.read_excel(ruta_datasets+'poblacion_provincias.xlsx')
superficie_provincias = pd.read_excel(ruta_datasets+'superficie_provincias.xlsx')

In [ ]:
df_concatenado = pd.concat([dataset_sube20,dataset_sube21,dataset_sube22,dataset_sube])

### Ejercicio 1

#### a)

Describimos el dataset

In [ ]:
dataset_sube.info()

Transformamos la columna dia_transporte a datetime

In [ ]:
dataset_sube['DIA_TRANSPORTE'] = pd.to_datetime(dataset_sube['DIA_TRANSPORTE'], format='%Y-%m-%d')

#### b)

##### i.

Creamos la columna fecha_dia con el nombre del día de la semana

In [ ]:
dias = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo']
def obtener_nombre_dia(fecha_dia):
    return dias[fecha_dia.weekday()]

In [ ]:
dataset_sube['FECHA_DIA'] = dataset_sube['DIA_TRANSPORTE'].apply(obtener_nombre_dia)

##### ii.

Creamos la columna fecha_ordinal donde asignamos el numero de día del año a cada fecha

In [ ]:
dataset_sube['FECHA_ORDINAL'] = dataset_sube['DIA_TRANSPORTE'].dt.day_of_year

##### iii.

Creamos la columna fecha_mes con el numero de mes para cada fecha

In [ ]:
dataset_sube['FECHA_MES'] = dataset_sube['DIA_TRANSPORTE'].dt.month

### Ejercicio 2

Creamos un dataframe que contiene solo los datos del AMBA y elimina los datos preliminares

In [ ]:
datos_amba = dataset_sube[dataset_sube['AMBA']=='SI']
datos_amba = datos_amba[datos_amba['DATO_PRELIMINAR']=='NO']

Acomodamos el orden de las columnas y les corregimos el formato del nombre

In [ ]:
datos_amba = datos_amba[['DIA_TRANSPORTE', 'FECHA_DIA', 'FECHA_MES','FECHA_ORDINAL', 'JURISDICCION', 'LINEA','CANTIDAD', 'TIPO_TRANSPORTE']]

In [ ]:
datos_amba = datos_amba.rename(columns={"DIA_TRANSPORTE": "fecha", "FECHA_DIA": "fecha_dia","FECHA_MES" : "fecha_mes", "FECHA_ORDINAL":"fecha_ordinal","JURISDICCION": "jurisdiccion", "LINEA":'linea',"CANTIDAD":"pasajeros", "TIPO_TRANSPORTE": "tipo_transporte"})

Resultado final:

In [ ]:
datos_amba.head()

### Ejercicio 3

#### a)

Calculamos la proporcion de pasajeros por transporte en 2023

In [ ]:
pasajeros_totales = sum(datos_amba['pasajeros'])

In [ ]:
pasajeros_por_transporte = datos_amba.groupby('tipo_transporte')['pasajeros'].sum()

In [ ]:
proporcion_pasajeros = ((pasajeros_por_transporte/pasajeros_totales)*100).round(1)
proporcion_pasajeros = proporcion_pasajeros.map(lambda x: f"{x}%")
proporcion_pasajeros

#### b)

Creamos un dataset con los datos del Subte, agrupamos por mes y linea, y obtenemos el valor maximo

In [ ]:
datos_subte = datos_amba[datos_amba['tipo_transporte'] == 'SUBTE']
datos_subte = datos_subte[['fecha_mes','linea','pasajeros']]

In [ ]:
datos_subte_mesLinea = datos_subte.groupby(['fecha_mes','linea'])['pasajeros'].sum()

In [ ]:
mes_linea_maxPasajeros = pd.DataFrame(datos_subte_mesLinea).idxmax()

Tupla (mes, línea de subte) donde viajó la mayor cantidad de pasajeros

In [ ]:
mes_linea_maxPasajeros.iloc[0]

#### c)

Creamos el dataset solo con los días habiles, calculamos el desvío estandar para cada día y vemos el menor

In [ ]:
dias_habiles = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes']
datos_amba_habiles = datos_amba[datos_amba['fecha_dia'].isin(dias_habiles)]

In [ ]:
desvios = []
for dia_habil in dias_habiles:
    desvio_estandar = datos_amba_habiles[datos_amba_habiles['fecha_dia']==dia_habil]['pasajeros'].std().round(1)
    desvio_dia = (dia_habil, desvio_estandar)
    desvios.append(desvio_dia)
print(desvios)

In [ ]:
dia_menos_desvio = min(desvios, key=lambda x: x[1])
dia_menos_desvio[0]

## Analisis exploratorio

### Ejercicio 4

##### Gráfico 1

In [ ]:
df_concatenado['DIA_TRANSPORTE'] = pd.to_datetime(df_concatenado['DIA_TRANSPORTE'], format='%Y-%m-%d')
df_concatenado['AÑO'] = df_concatenado['DIA_TRANSPORTE'].dt.year
so.Plot(data=df_concatenado, x="AÑO", y="CANTIDAD", color = "TIPO_TRANSPORTE").add(so.Bar(), so.Agg("sum"), so.Dodge()).scale(x=so.Nominal()).label(title="Cantidad de pasajeros por tipo de transporte por año", x="Año", y="Cantidad de pasajeros", color= "transporte" )

En este grafico podemos ver que el uso del transporte publico incrementa año tras año, si bien hay que tener en cuenta que en el año 2020 el uso de este bajo abruptamente debido a la pandemia de COVID-19, la cual se extendio hacia inicios del 2021 en muchos sectores. Las oficinas al igual que los establecimientos educativos adoptaron modos de trabajo virtual, lo que consecuentemente hizo que la poblacion tenga que salir menos de sus casas y por lo tanto tomar menos colectivos, subtes, trenes y lanchas. A partir del 2022 se ve un fuerte incremento del uso de dichos transportes con la pandemia ya dejada de lado, es probable que con los incrementos de los costos de mantener un vehiculo propio en los proximos años sigan subiendo los numeros de pasajeros.

##### Gráfico 2

In [ ]:
# Filtro los datos solo para lanchas
lanchas_data = df_concatenado[df_concatenado['TIPO_TRANSPORTE'] == 'LANCHAS']

# Creo una función para obtener las medianas por año
def obtener_medianas_por_año(data, year):
    data_año = data[data['AÑO'] == year]
    medianas = data_año.groupby('LINEA')['CANTIDAD'].median().reset_index()
    return medianas.sort_values(by='LINEA', ascending=False)

# Obtengo las medianas para cada año en especifico
lanchas_medianas20 = obtener_medianas_por_año(lanchas_data, 2020)
lanchas_medianas21 = obtener_medianas_por_año(lanchas_data, 2021)
lanchas_medianas22 = obtener_medianas_por_año(lanchas_data, 2022)
lanchas_medianas23 = obtener_medianas_por_año(lanchas_data, 2023)

# Creo una nueva figura y ejes para el gráfico combinado
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Gráfico 2020
axes[0, 0].bar(lanchas_medianas20['LINEA'], lanchas_medianas20['CANTIDAD'], color='b')
axes[0, 0].set_title('Mediana de pasajeros por línea de lancha (2020)')
axes[0, 0].set_xlabel('Línea de lancha')
axes[0, 0].set_ylabel('Cantidad de pasajeros (mediana)')
plt.setp(axes[0, 0].get_xticklabels(), rotation=45, ha='right')

# Gráfico 2021
axes[0, 1].bar(lanchas_medianas21['LINEA'], lanchas_medianas21['CANTIDAD'], color='g')
axes[0, 1].set_title('Mediana de pasajeros por línea de lancha (2021)')
axes[0, 1].set_xlabel('Línea de lancha')
axes[0, 1].set_ylabel('Cantidad de pasajeros (mediana)')
plt.setp(axes[0, 1].get_xticklabels(), rotation=45, ha='right')

# Gráfico 2022
axes[1, 0].bar(lanchas_medianas22['LINEA'], lanchas_medianas22['CANTIDAD'], color='r')
axes[1, 0].set_title('Mediana de pasajeros por línea de lancha (2022)')
axes[1, 0].set_xlabel('Línea de lancha')
axes[1, 0].set_ylabel('Cantidad de pasajeros (mediana)')
plt.setp(axes[1, 0].get_xticklabels(), rotation=45, ha='right')

# Gráfico 2023
axes[1, 1].bar(lanchas_medianas23['LINEA'], lanchas_medianas23['CANTIDAD'], color='purple')
axes[1, 1].set_title('Mediana de pasajeros por línea de lancha (2023)')
axes[1, 1].set_xlabel('Línea de lancha')
axes[1, 1].set_ylabel('Cantidad de pasajeros (mediana)')
plt.setp(axes[1, 1].get_xticklabels(), rotation=45, ha='right')

# Ajusto el espacio entre los subgráficos
plt.tight_layout()

# Muestro el gráfico
plt.show()

Estos gráficos no solo muestran cómo varió la mediana de pasajeros en las diferentes líneas de lancha durante los años 2020, 2021, 2022 y 2023, sino que también reflejan las posibles influencias externas que han afectado el uso del transporte acuático en los últimos años. En el gráfico del año 2020 se puede observar que la mediana es significativamente menor que la de los siguientes años ya que por la pandemia covid-19, la mayoría de personas no podían salir de sus casas y por ende se redujo el uso las lanchas (en el próximo gráfico analizaremos este interesante año más en detalle). En 2021, la mediana de pasajeros se estabilizó en comparación con 2020, las diferencias entre las líneas no son tan marcadas, lo que podría significar una distribución más uniforme del uso de las lanchas. Esto puede ser resultado de cambios en las políticas de transporte o en las rutas. En 2022, se nota un aumento en la mediana de pasajeros en varias líneas, esto nos muestra una posible recuperación en el uso de las lanchas. Este incremento podría estar relacionado con la reactivación económica y un regreso gradual a la normalidad post pandemia. En 2023, se puede ver un gran aumento en la mediana de la cantidad de pasajeros en comparación a los años anteriores, se podría decir que efectivamente se esta normalizando la movilidad y el uso de los transportes acuaticos despues de la pandemia.

##### Gráfico 3

In [ ]:
sns.barplot(data=datos_amba, x='fecha_dia', y='pasajeros', estimator='sum', order=dias, errorbar=None)

Este gráfico nos muestra la cantidad total de pasajeros en AMBA en los días de la semana. En conclusión, como mencionamos en la conclusión del gráfico anterior podemos observar que el Miércoles es el día en el que viaja la mayor cantidad de gente, seguido vienen los Martes y Jueves, asumimos que esto se debe a que luego de la pandemia y con el comienzo del trabajo virtual o tambien llamado home office, las empresas continuaron con esta metodología de trabajo híbrido (virtual y presencial) en la que los trabajadores eligen no ir presencial los Lunes y Viernes que son los dias pre y post fin de semana, esto se puede ver claramente reflejado en el uso del transporte público ya que estos días baja radicalmente el uso de la SUBE. Luego se puede ver que los sábados el uso del transporte publico se reduce pero la gran diferencia la marca el domingo, un dia en donde la gente prefiere quedarse en sus casas y no utilizar el transporte.

##### Gráfico 4

In [ ]:
pivot_table = datos_amba.pivot_table(
    values='pasajeros', 
    index='fecha_dia', 
    columns='tipo_transporte', 
    aggfunc='sum', 
    fill_value=0,
)

def thousands(x, pos):
    return '%1.0fK' % (x * 1e-3)

plt.figure(figsize=(10, 6))
heatmap = sns.heatmap(pivot_table, annot=True, fmt='d', cmap='viridis')
plt.title('Heatmap de Pasajeros por Tipo de Transporte y Día de la Semana')
plt.xlabel('Tipo de Transporte')
plt.ylabel('Día de la Semana')

for t in heatmap.texts:
    t.set_text(thousands(float(t.get_text()), None))
    
plt.show()

En este gráfico podemos observar la cantidad total de pasajeros en miles (K), por tipo de transporte y día de la semana, usando los datos de AMBA. Podemos resaltar que en los días de semana, es decir, los días laborales, hay mayor cantidad de pasajeros en todos los transportes públicos en comparación a los fines de semana. El día de mayor uso de transporte en general es el Miércoles pero esto lo vamos a analizar en un gráfico siguiente. Tambien tenemos que destacar la diferencia de cantdad de pasajeros entre los tipos de transporte, por ejemplo se nota fuertemente que la mayor cantidad de personas utilizan los colectivos en vez de subte o tren, esto se debe a que los colectivos tienen más lineas que los otros dos medios y que llegan a más lugares, como sabemos los subtes solo existen en CABA y aunque los trenes si los podemos encontrar en varias provincias, no en todas y esto afecta a la cantidad de usos de estos medios.

##### Gráfico 5

In [ ]:
agrupado_provincia = dataset_sube.groupby(['PROVINCIA'])['CANTIDAD'].sum().reset_index()

In [ ]:
agrupado_provincia = agrupado_provincia.merge(poblacion_provincias,how='left', on='PROVINCIA')

In [ ]:
agrupado_provincia_interior = agrupado_provincia[(agrupado_provincia['PROVINCIA']!='BUENOS AIRES') & (agrupado_provincia['PROVINCIA']!='JN')]

In [ ]:
plt.figure(figsize=(12, 6))
scatter = sns.scatterplot(data=agrupado_provincia_interior, x='POBLACION', y='CANTIDAD', hue='PROVINCIA', palette='viridis', legend=False)
for index, row in agrupado_provincia_interior.iterrows():
    scatter.text(row['POBLACION'], row['CANTIDAD'], row['PROVINCIA'], ha='left', fontsize=5, 
                 va='bottom', rotation_mode='anchor', 
                 bbox=dict(facecolor='white', alpha=0.5))
plt.xlabel('Población Total')
plt.ylabel('Cantidad de Pasajeros')
plt.title('Relación entre Pasajeros y Población por Provincia')
# plt.legend(title='Provincia', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

##### Gráfico 6

In [ ]:
# Creo una columna para el mes en el DataFrame del año 2020
dataset_sube20['DIA_TRANSPORTE'] = pd.to_datetime(dataset_sube20['DIA_TRANSPORTE'])
dataset_sube20['MES'] = dataset_sube20['DIA_TRANSPORTE'].dt.month

# Filtro los datos solo para lanchas
lanchas_data_2020 = dataset_sube20[dataset_sube20['TIPO_TRANSPORTE'] == 'LANCHAS']

# Obtengo la suma de pasajeros por mes
pasajeros_por_mes = lanchas_data_2020.groupby('MES')['CANTIDAD'].sum().reset_index()

# Creo el gráfico
plt.figure(figsize=(10, 6))
plt.bar(pasajeros_por_mes['MES'], pasajeros_por_mes['CANTIDAD'], color='r')
plt.title('Cantidad de pasajeros por mes en lancha - 2020')
plt.xlabel('Mes')
plt.ylabel('Cantidad de pasajeros')
plt.xticks(pasajeros_por_mes['MES'], ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre'], rotation=45)
plt.grid(axis='y')

# Muestro el gráfico
plt.tight_layout()
plt.show()

En este gráfico podemos observar mejor cómo afectó la pandemia en el uso de las lanchas. Más allá de que generalmente en los meses del verano (Enero, Febrero, Marzo) se utilizan las lanchas en mayor cantidad que en otros meses del año, ya que el clima es caluroso y la gente elige pasar tiempo en el rio, hay que tener en cuenta que en particular en el 2020, en el mes de Marzo inició la pandemia, por ende esto hace que la diferencia de la mediana de cantidad de pasajeros entre los meses del verano y los demás meses del año sea mayor que lo normal. Luego se puede observar un aumento gradual en la segunda mitad del año, esto podría indicar una recuperación en el uso de las lanchas a medida que las restricciones se reducian y las personas comenzaban a retomar sus actividades.

##### Gráfico 7

In [ ]:
datosSinBAJN = dataset_sube[(dataset_sube["PROVINCIA"] != "JN") & (dataset_sube["PROVINCIA"] != "BUENOS AIRES")]
datos_agrupados = datosSinBAJN.groupby(['PROVINCIA', 'TIPO_TRANSPORTE'])['CANTIDAD'].sum().unstack()

datos_agrupados.plot(kind='bar', stacked=True, figsize=(10, 6))
plt.title('Cantidad de pasajeros en colectivo por provincia (2023)')
plt.xlabel('Provincia')
plt.ylabel('Cantidad de pasajeros')
plt.xticks(rotation=45, ha='right')
plt.legend(title='Tipo de transporte')
plt.tight_layout()
plt.show()

En este gráfico podemos observar la cantidad total de pasajeros que viajaron en colectivo en las provincias de Argentina en 2023, excluyendo a Buenos Aires y JN ya que estos mostraban mucha diferencia en la cantidad de pasajeros y decidimos graficarlos aparte para así poder observar una distribución más equilibrada entre las demás provincias. En conclusión, podemos observar diferencias notables en el uso del colectivo en las diferentes provincias. Las provincias con mayor cantidad de pasajeros en colectivo probablemente cuentan con una mejor infraestructura y accesibilidad en términos de rutas y disponibilidad de transporte público. Podemos notar que en Córdoba no hay ningun registro de uso de la sube en 2023, luego de hacer una busqueda logramos descubrir que en esta provincia se implementó la SUBE a principios del 2024, por lo tanto no hay datos al respecto. Podemos destacar Mendoza, la cual posee mayor uso de la SUBE en comparación a las demas provincias, esto puede deberse a varios factores como la cantidad de poblacion que vive en la provincia, una infraestructura de transporte público más desarrollada, mayor apoyo gubernamental y subsidios.

##### Gráfico 8

In [ ]:
datosSoloBAJN = dataset_sube[dataset_sube["TIPO_TRANSPORTE"]=="COLECTIVO"]
datosSoloBAJN = datosSoloBAJN[(datosSoloBAJN["PROVINCIA"] == "JN") | (dataset_sube["PROVINCIA"] == "BUENOS AIRES")]
datos_agrupados2 = datosSoloBAJN.groupby(['PROVINCIA', 'TIPO_TRANSPORTE'])['CANTIDAD'].sum().unstack()

datos_agrupados2.plot(kind='bar', stacked=True, figsize=(5, 5))
plt.title('Cantidad de pasajeros en colectivos en BUENOS AIRES Y JN (2023)')
plt.xlabel('Provincia')
plt.ylabel('Cantidad de pasajeros')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

El gráfico de barras muestra la cantidad total de pasajeros que viajaron en colectivos en la provincia de Buenos Aires y en Jurisdicción Nacional durante el año 2023. Para este analizis hay que tener en cuenta que Buenos Aires es la capital de Argentina y uno de los centros más importantes del país, donde vive una gran parte de la población y donde se realizan gran parte de las actividades económicas, esto se refleja en la alta demanda de transporte público. La Jurisdicción Nacional, incluye áreas bajo jurisdicción directa del gobierno federal, puede presentar características específicas en cuanto a la distribución de la población, el tipo de actividades y la disponibilidad de transporte público. Estas caracteristicas hacen posiblemente que sea menor la cantidad de pasajeros en colectivos en comparación con Buenos Aires. Además queremos agregar que observamos que la linea BS_AS_LINEA_514 no tiene asignada ninguna provincia (nan), luego de hacer una investigacion, en el dataset exportado desde la página correspondiente vemos que esta linea tiene asignada la provincia de "BUENOS AIRES", igualmente teniendo en cuenta esta diferencia en los datos, podemos afirmar que esto no cambia nuestras conclusiones anteriores ya que se sumarian más datos a la provincia de Buenos Aires que es la que tiene mayor cantidad de información.

In [ ]:
#Reviso si hay datos que en la columna de provincia sean nan y que no sean subte ya que se que estos no tienen asignada ninguna provincia
dataset_sube[(dataset_sube["PROVINCIA"].isna()) & (dataset_sube["TIPO_TRANSPORTE"] != "SUBTE")]

## Modelado

### Ejercicio 5

Creamos el dataset para los modelos y hacemos preprocesamiento

In [ ]:
datos_ColectivoJN = dataset_sube_regresion[(dataset_sube_regresion['TIPO_TRANSPORTE']=='COLECTIVO') & (dataset_sube_regresion['PROVINCIA']=='JN') & (dataset_sube_regresion['AMBA']=='SI')]
cols = datos_ColectivoJN.LINEA.unique()

In [ ]:
pasajeros_por_linea = pd.DataFrame()
for col in  cols:
    datos_linea = datos_ColectivoJN[datos_ColectivoJN.LINEA ==col][["DIA_TRANSPORTE", "CANTIDAD"]]
    datos_linea =datos_linea.set_index("DIA_TRANSPORTE").rename(columns ={"CANTIDAD" : col})
    pasajeros_por_linea = pd.concat([pasajeros_por_linea ,datos_linea], axis = 1)

In [ ]:
pasajeros_por_linea = pasajeros_por_linea.dropna(axis=1)

In [ ]:
pasajeros_por_linea.head()

#### Regresion Lineal Univariada

In [ ]:
columna_regresion_univ = top_5_columns[0]

In [ ]:
predictor_univ = pasajeros_por_linea[[columna_regresion_univ]]

In [ ]:
modelo_univ = linear_model.LinearRegression()    
modelo_univ.fit(predictor_univ, pasajeros_por_linea[['BSAS_LINEA_009']])

In [ ]:
y_pred_univ = modelo_univ.predict(predictor_univ)
r2_score(pasajeros_por_linea[['BSAS_LINEA_009']], y_pred_univ)

In [ ]:
mse = mean_squared_error(pasajeros_por_linea[['BSAS_LINEA_009']], y_pred_univ)
mae = mean_absolute_error(pasajeros_por_linea[['BSAS_LINEA_009']], y_pred_univ)
r2 = r2_score(pasajeros_por_linea[['BSAS_LINEA_009']], y_pred_univ)

print(f'Error Cuadrático Medio: {np.sqrt(mse)}')
print(f'Error Absoluto Medio: {mae}')
print(f'R2: {r2}')

#### Regresion Lineal Multivariada

En este caso, elejimos 5 columnas para entrenar el modelo en base a la varianza de los datos. Primero calculamos la varianza de la variable a predecir, Linea 009, y despues buscamos las 5 columnas que tengan varianza simialar a ésta.

In [ ]:
varianzas = pasajeros_por_linea.var()

varianza_009 = varianzas['BSAS_LINEA_009']

diferencia_varianza = np.abs(varianzas - varianza_009)

columnas_mas_cercanas = diferencia_varianza.nsmallest(6).index  
columnas_mas_cercanas = columnas_mas_cercanas[columnas_mas_cercanas != 'BSAS_LINEA_009'] 
print(f'Lineas con las varianza mas cercana a la 009: {columnas_mas_cercanas.values}')


In [ ]:
modelo_multivariado = linear_model.LinearRegression() 

In [ ]:
X_mv=pasajeros_por_linea[columnas_mas_cercanas]
y_mv=pasajeros_por_linea['BSAS_LINEA_009']
X_train_mv, X_test_mv, y_train_mv, y_test_mv = train_test_split(X_mv, y_mv, test_size=0.2, random_state=42)

In [ ]:
modelo_multivariado.fit(X_train_mv, y_train_mv)

In [ ]:
y_pred_mv = modelo_multivariado.predict(X_test_mv)

In [ ]:
y_test_mv = y_test_mv.values.ravel()

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(x=y_test_mv, y=y_pred_mv)
plt.plot([y_test_mv.min(), y_test_mv.max()], [y_test_mv.min(), y_test_mv.max()], 'k--', lw=3)
plt.xlabel('Valores Reales')
plt.ylabel('Valores Predichos')
plt.title('Regresión Lineal Multivariada')
plt.show()

In [ ]:
mse = mean_squared_error(y_test_mv, y_pred_mv)
mae = mean_absolute_error(y_test_mv, y_pred_mv)
r2 = r2_score(y_test_mv, y_pred_mv)

print(f'Error Cuadrático Medio: {np.sqrt(mse)}')
print(f'Error Absoluto Medio: {mae}')
print(f'R2: {r2}')

#### Regresion Ridge

Para este modelo vamos a calcular la correlación de todas las variables del dataset con nuestra variable a predecir, y vamos a tomar las 5 primeras para entrenar el modelo.

In [ ]:
matriz_correlacion = pasajeros_por_linea.corr()
correlaciones = matriz_correlacion['BSAS_LINEA_009'].abs().sort_values(ascending=False)

top_5_lineas = correlaciones.index[1:6] 

print("Lineas con mejor correlación con Linea 009:", top_5_lineas)

In [ ]:
dataset_modelo_ridge = pasajeros_por_linea[top_5_lineas.tolist() + ['BSAS_LINEA_009']]

In [ ]:
dataset_modelo_ridge = dataset_modelo_ridge.rename(columns={'LINEA 101':'LINEA_101', 'LINEA 7':'LINEA_7', 'LINEA 107': 'LINEA_107'})

In [ ]:
df_train, df_test = train_test_split(dataset_modelo_ridge, test_size=0.2, random_state=42)

formula = 'BSAS_LINEA_009 ~ (LINEA_101+BSAS_LINEA_146+LINEA_7+BSAS_LINEA_024+LINEA_107)'
y_ridge, X_ridge =  Formula(formula).get_model_matrix(df_train)

Proponemos alphas arbitrarios para el modelo

In [ ]:
alphas = np.array([0.1, 0.3, 0.5, 1, 3, 7])

In [ ]:
cv = KFold(n_splits=5, random_state=42, shuffle=True) 
for alpha in alphas:
    modeloRidge = linear_model.Ridge(alpha = alpha, fit_intercept = False)    
    rmse = np.zeros(cv.get_n_splits()) 

    ind = 0
    
    for train_index, val_index in cv.split(X_ridge):
        X_train, X_val, y_train, y_val = X_ridge.iloc[train_index], X_ridge.iloc[val_index], y_ridge.iloc[train_index], y_ridge.iloc[val_index]
        modeloRidge.fit(X_train, y_train)

        y_pred = modeloRidge.predict(X_val)
        rmse[ind] = np.sqrt(mean_squared_error(y_val, y_pred))
        ind = ind + 1

    print(f"Para alpha = {alpha:.5f} la raíz del Error Cuadratico Medio es: {rmse.mean():.5f}")

Hacemos una segunda corrida con valores cercanos al 7.

In [ ]:
alphas = np.array([5, 7, 9, 11, 13, 15])

for alpha in alphas:
    modeloRidge = linear_model.Ridge(alpha = alpha, fit_intercept = False)    
    rmse = np.zeros(cv.get_n_splits()) 

    ind = 0

    for train_index, val_index in cv.split(X_ridge):
        X_train, X_val, y_train, y_val = X_ridge.iloc[train_index], X_ridge.iloc[val_index], y_ridge.iloc[train_index], y_ridge.iloc[val_index]
        modeloRidge.fit(X_train, y_train)

        y_pred = modeloRidge.predict(X_val)
        rmse[ind] = np.sqrt(mean_squared_error(y_val, y_pred))
        ind = ind + 1

    print(f"Para alpha = {alpha:.5f} la raiz del error cuadratico medio es: {rmse.mean():.5f}")


In [ ]:
alpha_optimo = 15

modeloRidge = linear_model.Ridge(alpha = alpha_optimo, fit_intercept = False)    
modeloRidge.fit(X_ridge, y_ridge)

In [ ]:
y_test_ridge, X_test_ridge =  Formula(formula).get_model_matrix(df_test)
        
y_pred_ridge = modeloRidge.predict(X_test_ridge)

In [ ]:
y_test_ridge_array = np.ravel(y_test_ridge)
y_pred_ridge_array = np.ravel(y_pred_ridge)

plt.figure(figsize=(10, 6))

sns.scatterplot(x=y_test_ridge_array, y=y_pred_ridge_array)
plt.plot([y_test_ridge_array.min(), y_test_ridge_array.max()], [y_test_ridge_array.min(), y_test_ridge_array.max()], 'k--', lw=3)
plt.xlabel('Valores Reales')
plt.ylabel('Valores Predichos')
plt.title('Regresión Ridge')

plt.show()

In [ ]:
mse = mean_squared_error(y_test_ridge, y_pred_ridge)
mae = mean_absolute_error(y_test_ridge, y_pred_ridge)
r2 = r2_score(y_test_ridge, y_pred_ridge)

print(f'Error Cuadrático Medio: {np.sqrt(mse)}')
print(f'Error Absoluto Medio: {mae}')
print(f'R2: {r2}')

In [ ]:
coefs = modeloRidge.coef_[0].tolist()
coefs = coefs[1:]

In [ ]:
formula_Ridge = f'Y = {intercept:.1f} '
for i in range(len(top_5_lineas)):
    formula_Ridge += f'+ ({coefs[i]:.2} * {top_5_lineas[i]}) '

formula_Ridge